In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle


curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:

with open("intents.json") as file:
    data = json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'context_set': ''},
  {'tag': 'age',
   'patterns': ['how old',
    'how old is Dheeyush',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 22 years old!', '22 years young!'],
   'context_set': ''},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Dheeyush.',
    "I'm Dheeyush!",
    "I'm Dheeyush the bot."],
   'context_set': ''},
  {'tag': 'shop',
   'patterns': ['Id like to buy something',
    'whats on t

In [4]:
# Data Preprocessing

In [5]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [6]:
print(len(words))
words

46


['a',
 'ag',
 'am',
 'anyon',
 'ar',
 'buy',
 'cal',
 'could',
 'cya',
 'day',
 'dheeyush',
 'do',
 'eat',
 'get',
 'good',
 'goodby',
 'guy',
 'hav',
 'hello',
 'hi',
 'hour',
 'how',
 'i',
 'id',
 'is',
 'lat',
 'leav',
 'lik',
 'menu',
 'nam',
 'of',
 'old',
 'on',
 'op',
 'reccommend',
 'see',
 'should',
 'someth',
 'the',
 'ther',
 'to',
 'up',
 'what',
 'when',
 'yo',
 'you']

In [7]:
print(len(labels))
labels

6


['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']

In [8]:
print(len(training))
training

26


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
print(len(output))
output


26


array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [10]:
# model a neural network

In [12]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    # uncomment for first time use
    model.load('chatbot.tflearn')
except Exception as e:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save('chatbot.tflearn')

Training Step: 3999  | total loss: 0.03150 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.03150 - acc: 0.9998 -- iter: 24/26
Training Step: 4000  | total loss: 0.03157 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.03157 - acc: 0.9999 -- iter: 26/26
--
INFO:tensorflow:C:\Users\Dheeraj\Desktop\deep learning chatbot\chatbot.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        print(results)
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print(tag)
        
        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            print("I didn't get that, try again or ask another question!!!...")

In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: hello
[1.5736744e-02 7.8935316e-03 9.7632217e-01 8.5241794e-07 3.5562345e-05
 1.1198760e-05]
greeting
Good to see you again!
You: hello
[1.5736744e-02 7.8935316e-03 9.7632217e-01 8.5241794e-07 3.5562345e-05
 1.1198760e-05]
greeting
Hello!
